In [38]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from sklearn.linear_model import LinearRegression
from plotly.subplots import make_subplots
import plotly.express as px
from scipy.stats import pearsonr, spearmanr
import numpy as np
import plotly.express as px

In [32]:
def sk_linreg(df, coef=False, mean=False):
    df.dropna(inplace=True)
    df_linreg = pd.DataFrame()
    for region in np.unique(df['Region']):
        df_ = df[df['Region']==region].copy()
        df_.dropna(inplace=True)
        X = np.array(df_['Year']).reshape(-1, 1)
        if mean:
            df_.iloc[:,-1] = df_.iloc[:,-1] - df_.iloc[:,-1].mean()
        reg = LinearRegression().fit(X, df_.iloc[:,-1])
        df_['Linear Regression Line'] = reg.predict(X)
        df_linreg = pd.concat([df_linreg, df_])
    if coef:
        return df_linreg, reg.coef_
    return df_linreg

def plot(df_, region=None):
    if region == 'Davos':
        df_ = df_[df_['Region']=='Davos'].copy()
        df_.dropna(inplace=True)
    data_name = df_.columns[-1]
    fig = go.Figure()
    for region in np.unique(df_['Region']):
        df = df_[df_['Region']==region]
        fig.add_trace(go.Scatter(x=df['Year'], y=df[data_name],
                                 name=region, line=dict(color='red')
                                 ))

    fig.update_layout(title=data_name + ' pro Jahr' + ' in Davos' if region=='Davos' else '',
                   xaxis_title='Jahr',
                    template='simple_white',
                   yaxis_title=data_name + ' [cm]')
    return fig

def plot_linreg(df, data_name='Neuschnee'):
    df.dropna(inplace=True)
    df, coef = sk_linreg(df, True)
    fig = go.Figure()
    unit = '[cm]' if data_name=='Neuschnee' else ''
    fig.add_trace(go.Scatter(x=df['Year'], y=df[data_name], name=data_name, line=dict(color='red')))
    fig.add_trace(go.Scatter(x=df['Year'], y=df['Linear Regression Line'], name='Linear Regression'))
    fig.update_layout(title='{} pro Jahr. Veränderung von {:.2} {} pro Jahr'.format(data_name, coef[0], unit),
                      xaxis_title='Jahr',
                      showlegend=False,
                      template='simple_white',
                      yaxis_title=data_name + ' ' + unit)
    return fig

def plot_linreg_subplots(df, data_name='Neuschnee'):
    coefs = []
    for region in np.unique(df['Region']):
        df_ = df[df['Region']==region].copy()
        df_, coef = sk_linreg(df_, True)
        coefs.append(coef[0])
    
    subplots_titles = []
    for coef, region in zip(coefs, np.unique(df['Region'])):
        subplots_titles.append('{} in {}. Veränderung von {:.2} pro Jahr.'.format(data_name, region, coef))
    
    nr_region = len(np.unique(df['Region']))
    fig = make_subplots(rows=nr_region, cols=1, subplot_titles=subplots_titles)
    
    for i, region in enumerate(np.unique(df['Region']), 1):

        df_ = df[df['Region']==region].copy()
        df_, coef = sk_linreg(df_, True)
        fig.append_trace(go.Scatter(x=df_['Year'], y=df_[data_name],
                                    name=data_name),
                         row = i, col = 1)
        fig.append_trace(go.Scatter(x=df_['Year'], y=df_['Linear Regression Line'], name=np.round(coef[0],2)),
                         row = i, col = 1)
    
    
    fig.update_layout(height=3000, width=600, title_text=data_name+ " und Lineare Regression.",
                      showlegend=False, xaxis_title='Jahr', template='simple_white',
                      yaxis_title=data_name)
    return fig


def calculate_corr(df1, df2):
    df1.dropna(inplace=True)
    df2.dropna(inplace=True)
    corr_ps = []
    corr_sm = []
    regions = []

    for region in np.unique(df1['Region']):
        df1_ = df1[df1['Region']==region].copy()
        df2_ = df2[df2['Region']==region].copy()
        first_year = max(df1_['Year'].min(), df2_['Year'].min())
        df1_ = df1_[df1_['Year']>=first_year].copy()
        df2_ = df2_[df2_['Year']>=first_year].copy()
        df1_.sort_values(by='Year', inplace=True)
        df2_.sort_values(by='Year', inplace=True)
        try:
            corr_ps_, _ = pearsonr(df1_.iloc[:,-1], df2_.iloc[:,-1])
            corr_sm_, _ = spearmanr(df1_.iloc[:,-1], df2_.iloc[:,-1])
            corr_ps.append(corr_ps_)
            corr_sm.append(corr_sm_)
            regions.append(region)
        except Exception as E:
            print(E, "with ", region)

    corr_df = pd.DataFrame(list(zip(regions,corr_ps, corr_sm)), 
                           columns=['Region', 'Pearson Correlation', 'Spearman Correlation'])
    return corr_df


def lineplot_swiss_temp(data, data_name = 'Neuschnee'):
    fig = go.Figure()
    for region in np.unique(data['Region']):
        df_ = data[data['Region']==region].copy()
        df_[data_name] = df_[data_name] - df_[data_name].mean()
        if region != 'Davos':
            fig.add_trace(go.Scatter(x=df_['Year'],
                                     y=df_[data_name],
                                     name=data_name,
                                     line=dict(color='grey'),
                                     opacity=0.3)
                         )
        else:
            fig.add_trace(go.Scatter(x=df_['Year'],
                                     y=df_[data_name],
                                     name=data_name,
                                     line=dict(color='red',
                                               width=2)
                                    )
                         )
    fig.add_trace(go.Scatter(x=data['Year'],
                             y=data[data_name]*0,
                             name='Zentriertes Mittel',
                             line=dict(color='black',
                                       dash='dot',
                                       width=1),
                             opacity=0.8)
                 )
    fig.update_layout(template='simple_white',
                      title='Temperaturabweichungen aller Orte zu ihrem Mittelwert',
                      showlegend=False,
                      yaxis_title='Temperaturdifferenz [K]',
                      xaxis_title='Jahre')
    fig.show()


def selectable_region_plot(df, data_name = 'Neuschnee'):
    df = sk_linreg(df, False, True).copy()
    list_updatemenus = []
    nr_region = len(df['Region'])
    data = []
    for i, region in enumerate(np.unique(df['Region'])):
        visible_ = [False] * 2* nr_region
        visible_[2*i:2*i+2] = [True, True]
        dict_ = {'label': region,
                'method': 'update',
                'args': [{'visible': visible_,
                         'title': 'Neuschnee [cm] pro Jahr in ' + region}]
                }
        list_updatemenus.append(dict_)
        df_ = df[df['Region']==region]
        data.append(go.Scatter(x=df_['Year'],
                               y=df_[data_name] - df_[data_name].mean(),
                               name=region,
                               line=dict(color='grey',
                                         width=0.8) if region != 'Davos' else
                               dict(color='red',
                                         width=2)
                   ))
        data.append(go.Scatter(x=df_['Year'],
                               y=df_['Linear Regression Line'],
                               name=region + ' LR',
                               line=dict(color='black',
                                         dash='dot',
                                         width=1),
                               opacity=0.8)
                   )

    layout=go.Layout(title=data_name + 'extremas in verschiedenen Regionen',updatemenus=list([dict(buttons=list_updatemenus)]),
                     showlegend=False, template='simple_white', barmode='overlay')#defining figure and plotting
    fig = go.Figure(data,layout)
    return fig


In [23]:
df = pd.read_csv('Neuschnee.csv')
np.unique(df['Region'])

array(['Basel-Binningen', 'Bern-Zollikofen', 'Davos', 'Genf-Cointrin',
       'Locarno-Monti', 'Lugano', 'Luzern', 'Neuenburg', 'Samedan',
       'Sitten', 'St. Gallen', 'Säntis', 'Zürich-Fluntern'], dtype=object)

# Neuschnee Davos
Meine Grossmutter geht seit vielen Jahren jährlich zum Skifahren nach Davos. Sie behauptet, dass in den letzten Jahren
weniger Schnee hat. Nach dieser Aussage habe ich ein bisschen nachgedacht und mir ist auch aufgefallen, dass es im letzten
Jahr viele Flächen ohne Schnee auf der Piste hatte. Mit der Neugier im Gepäck, habe ich nach passenden Daten gesucht
und diese auf [Swissopendata.ch](https://opendata.swiss/de/dataset/klimadaten-sonnenscheindauer-niederschlag-temperatur-und-neuschnee2)
gefunden.
Das ist der jährliche Neuschnee in Davos.

In [24]:
df = pd.read_csv('Neuschnee.csv')
fig = plot(df, 'Davos')
fig.show()

Hier habe ich gesehen, dass es so scheint, als gäbe es einen Abwärtstrend. Mit einer Linearen Regressions (Ordinary Least Squares)
lässt sich das auch "wissenschaftlich" veranschaulichen und man sieht, dass es eine Abnahme gibt, von fast einem cm pro Jahr.
Um die Abnahme deutlicher darzustellen, zeigen wir nur die Extremas, dass heisst, wir ziehen von jedem Wert
den Durchschnitt ab von den gesamten Daten.

In [25]:
df = pd.read_csv('Neuschnee.csv')
df = df[df['Region']=='Davos']
plot_linreg(df).show()

In den anderen Regionen sieht man diese Abnahme ebenfalls. In diesem Plot kann man die einzelnen
Regionen anschauen mit einer Linearen Regression. Davos ist markiert als rote Linie und die Linie der linearen Regression
als gestrichelte Linie.

In [26]:
df = pd.read_csv('Neuschnee.csv')
selectable_region_plot(df)

Man sieht ein überall eine Abnahme des gemessenen Neuschnees, ausser in Samedan, wo es eine Zunahme sieht. Wenn man
den Durchschnitt des gefallenen Neuschnees in der Schweiz nimmt, sieht man eine gesamthafte Abnahme des Neuschnees.

In [27]:
df = pd.read_csv('Neuschnee.csv')
df['Region'] = 'Schweiz'
df = df.groupby(['Year', 'Region'], as_index=False).mean()
plot_linreg(df)

Da wir im Datensatz auch die Temperatur geliefert bekommen haben, haben wir die Entwicklung des Neuschnees mit der
Temperatur verglichen. Bei der Temperatur sieht man einen klaren Aufwärtstrend. Seit Messbegin 1931 ist die Temperatur in der Schweiz um ca. 2.9 C° gestiegen. 

In [28]:
df = pd.read_csv('Jahrestemperatur.csv')
selectable_region_plot(df, data_name = 'Jahrestemperatur')

Wenn man die Durchschnitstemperatur (y-Achse) mit dem Durchschnittsneuschnee (x-Achse) darstellt, sieht man erstens eine Abnahme und zweitens, da das Jahr die Farbe bestimmt, dass viele sehr warme Jahre in den letzten 30 Jahren stattgefunden haben.

In [53]:
df = pd.read_csv('Jahrestemperatur.csv')
df = df.groupby('Year', as_index=False).mean()
df_neuschnee = pd.read_csv('Neuschnee.csv')
df_neuschnee = df_neuschnee.groupby('Year', as_index=False).mean()
df = df.merge(df_neuschnee, on ='Year')

fig = px.scatter(df, x="Neuschnee", y="Jahrestemperatur", color="Year",
                hover_data=['Jahrestemperatur'], trendline="ols")
fig.show()

Mit Hilfe der Pearsoncorrelation lässt sich auch zeigen, dass die der Neuschnee und die Jahrestemperatur sehr stark invers korrelieren. Desto näher der Wert bei 1 (-1) sieht, desto stärker (invers) korrelieren zwei Datensätze miteinander.

In [59]:
df_jahresniederschlag = pd.read_csv('Jahrestemperatur.csv')
df_neuschnee = pd.read_csv('Neuschnee.csv')
corr_regen_schnee = calculate_corr(df_jahresniederschlag, df_neuschnee)
fig = px.bar(corr_regen_schnee, x='Region', y='Pearson Correlation', title='Korrelation Temperatur und Neuschnee')
fig.show()

x and y must have the same length. with  Luzern
